In [1]:
import pandas as pd
import numpy as np
from path import Path
import torch.utils.data as data
from imageio import imread
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F 
import matplotlib.pyplot as plt
%matplotlib inline 
from torch.utils.data.sampler import SubsetRandomSampler

from PIL import *
import ast

In [2]:
import numpy as np

class ScaledDotProductAttention(nn.Module):
    ''' Scaled Dot-Product Attention '''
    # given query, key,value it finds the rightful weighted component of v to get the attention applied ouput
    #q,v,k- batch X length of sequence X features or encoding
    #attention sholuld be -batchX7X7
    def __init__(self, temperature, attn_dropout=0.1):
        super().__init__()
        self.temperature = temperature
        self.dropout = nn.Dropout(attn_dropout)
        self.softmax = nn.Softmax(dim=2)

    def forward(self, q, k, v, mask=None):
#         print(k.transpose(1,2).shape)

        attn = torch.bmm(q, k.transpose(1, 2)) 
#         print(attn.shape)
        attn = attn / self.temperature

        if mask is not None:
            attn = attn.masked_fill(mask, -np.inf)

        attn = self.softmax(attn)
        attn = self.dropout(attn)
#         print(str(attn.shape)+" "+str(v.shape))
        output = torch.bmm(attn, v)

        return output, attn
SDP=ScaledDotProductAttention(5)
Ss=SDP(torch.zeros(5,6,100),torch.zeros(5,6,100),torch.zeros(5,6,100))
# print(Ss[0].shape)
class MultiHeadAttention(nn.Module):
    ''' Multi-Head Attention module '''

    def __init__(self, n_head, d_model, d_k, d_v, dropout=0.1):
        super().__init__()

        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v

        self.w_qs = nn.Linear(d_model, n_head * d_k)
        self.w_ks = nn.Linear(d_model, n_head * d_k)
        self.w_vs = nn.Linear(d_model, n_head * d_v)
        nn.init.normal_(self.w_qs.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_k)))
        nn.init.normal_(self.w_ks.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_k)))
        nn.init.normal_(self.w_vs.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_v)))

        self.attention = ScaledDotProductAttention(temperature=np.power(d_k, 0.5))
        self.layer_norm = nn.LayerNorm(d_model)

        self.fc = nn.Linear(n_head * d_v, d_model)
        nn.init.xavier_normal_(self.fc.weight)

        self.dropout = nn.Dropout(dropout)


    def forward(self, q, k, v, mask=None):

        d_k, d_v, n_head = self.d_k, self.d_v, self.n_head

        sz_b, len_q, _ = q.size()
        sz_b, len_k, _ = k.size()
        sz_b, len_v, _ = v.size()
#         print(str(sz_b)+"die")

        residual = q

        q = self.w_qs(q).view(sz_b, len_q, n_head, d_k)
        k = self.w_ks(k).view(sz_b, len_k, n_head, d_k)
        v = self.w_vs(v).view(sz_b, len_v, n_head, d_v)

        q = q.permute(2, 0, 1, 3).contiguous().view(-1, len_q, d_k) # (n*b) x lq x dk
        k = k.permute(2, 0, 1, 3).contiguous().view(-1, len_k, d_k) # (n*b) x lk x dk
        v = v.permute(2, 0, 1, 3).contiguous().view(-1, len_v, d_v) # (n*b) x lv x dv
#         print("v-"+str(v.shape))
        if mask is not None:
            mask = mask.repeat(n_head, 1, 1) # (n*b) x .. x ..
        output, attn = self.attention(q, k, v, mask=mask)
#         print(q.shape,k.shape,v.shape)
        output = output.view(n_head, sz_b, len_q, d_v)
        output = output.permute(1, 2, 0, 3).contiguous().view(sz_b, len_q, -1) # b x lq x (n*dv)

        output = self.dropout(self.fc(output))
        output = self.layer_norm(output + residual)

        return output, attn
MHA=MultiHeadAttention(4,15,15,15)
op=MHA(torch.zeros(5,7,15),torch.zeros(5,7,15),torch.zeros(5,7,15))
# print(op[0].shape)
class PositionwiseFeedForward(nn.Module):
    ''' A two-feed-forward-layer module '''

    def __init__(self, d_in, d_hid, dropout=0.1):
        super().__init__()
        self.w_1 = nn.Conv1d(d_in, d_hid, 1) # position-wise
        self.w_2 = nn.Conv1d(d_hid, d_in, 1) # position-wise
        self.layer_norm = nn.LayerNorm(d_in)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        residual = x
        output = x.transpose(1, 2)
#         print("FCC-"+str(output.shape))
#         print("FFC_out-"+str(self.w_1(output).shape))
        output = self.w_2(F.relu(self.w_1(output)))
        output = output.transpose(1, 2)
        output = self.dropout(output)
        output = self.layer_norm(output + residual)
        return output

    

In [19]:
class EncoderLayer(nn.Module):
    ''' Compose with two layers '''

    def __init__(self, d_model, d_inner, n_head, d_k, d_v, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.slf_attn = MultiHeadAttention(
            n_head, d_model, d_k, d_v, dropout=dropout)
        self.pos_ffn = PositionwiseFeedForward(d_model, d_inner, dropout=dropout)

    def forward(self, enc_input, non_pad_mask=None, slf_attn_mask=None):
        enc_output, enc_slf_attn = self.slf_attn(
            enc_input, enc_input, enc_input, mask=slf_attn_mask)
#         print("ENC_o")
#         print(enc_output.shape)


        enc_output = self.pos_ffn(enc_output)


        return enc_output, enc_slf_attn
    
XX=EncoderLayer(15,10,4,10,10)

zz=XX(torch.zeros(5,7,15))
# print("ENc")
# print(zz[0].shape)
# print("start")
class Encoder(nn.Module):
    ''' A encoder model with self attention mechanism. '''

    def __init__(self,n_modality,d_model,n_head,d_k,d_v,dropout,n_layers,d_inner=500):
        #d_model - number of features in input 100 here
        #n_head - number of heads of multihaded attention
        #d_k=d_q=  number of features in query, key
        #d_v = number of features in value whose weighted(attentioned) sum we gonna take
        

        super().__init__()
        self.n_modality=n_modality
#         self.stn=nn.ModuleList([SpatialTransformer(3, (240,240), 8) for _ in range(n_ref)])
        
        self.layer_stack = nn.ModuleList([EncoderLayer(d_model, d_inner, n_head, d_k, d_v, dropout=dropout) 
                                          for _ in range(n_layers)])
        #self.em=nn.Linear(225,100)
        self.fc1=nn.Linear(d_model*n_modality,300)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(300,100)
        self.fc3=nn.Linear(100,3)
#         self.fc4=nn.Linear(50,3)
        self.bn1 = nn.BatchNorm1d(num_features=300)
        self.bn2 = nn.BatchNorm1d(num_features=100)
        self.softmax=nn.Softmax(1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, embeddings1,embeddings2 ):


        
        encodings_total=[embeddings1,embeddings2]

        enc_output=torch.stack(encodings_total,0)

        
        enc_output=enc_output.permute(1,0,2)
#         print("encoding_OUTPUT2-"+str(enc_output.shape))

        for enc_layer in self.layer_stack:
            enc_output, enc_slf_attn = enc_layer(enc_output,non_pad_mask=None,slf_attn_mask=None)
           
        
        final_input=enc_output.reshape(enc_output.shape[0],-1)

        final=self.relu(self.fc3(self.bn2(self.relu((self.fc2(self.bn1(self.relu(self.fc1(final_input)))))))))
        
        return(final)
     

In [4]:
e1=torch.zeros(4,100)
e2=torch.zeros(4,100)
e3=torch.stack([e1,e2],0)
e3.shape

torch.Size([2, 4, 100])

In [20]:
feature_set_MLP=pd.read_csv('BCLL_MLP_features_ankit.csv')
feature_set_LSTM=pd.read_csv('BCLL_LSTM_features_ankit.csv')
feature_set_MLP.head()

,gene_name,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_90,tag_91,tag_92,tag_93,tag_94,tag_95,tag_96,tag_97,tag_98,tag_99
0,hint1,125133.351562,357573.156250,0.000000,587084.000000,0.0,315726.906250,0.0,175430.843750,268782.625000,...,442658.781250,0.0,465458.937500,370437.156250,0.0,0.0,472186.937500,0.0,0.0,466427.312500
1,rabggta,20101.933594,62892.046875,0.000000,103993.031250,0.0,55333.929688,0.0,30097.763672,48279.539062,...,79100.062500,0.0,82092.179688,65288.472656,0.0,0.0,83334.328125,0.0,0.0,81504.468750
2,ywhae,6624.807129,14475.721680,0.000000,24332.548828,0.0,13422.669922,0.0,9587.588867,10549.949219,...,18221.716797,0.0,19292.927734,14790.755859,0.0,0.0,19355.056641,0.0,0.0,19177.373047
3,kat2b,4142.221680,8753.067383,0.000000,14703.916016,0.0,8161.906250,0.0,5902.970215,6346.573730,...,11016.152344,0.0,11690.555664,8958.273438,0.0,0.0,11691.131836,0.0,0.0,11626.661133
4,smad5,1322.737061,2288.863037,26.059177,3835.314453,0.0,2206.577148,0.0,1791.832397,1568.654663,...,2822.048828,0.0,3072.452148,2335.262207,0.0,0.0,3047.292969,0.0,0.0,3087.742920


In [21]:
feature_set_MLP.head()

,gene_name,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_90,tag_91,tag_92,tag_93,tag_94,tag_95,tag_96,tag_97,tag_98,tag_99
0,hint1,125133.351562,357573.156250,0.000000,587084.000000,0.0,315726.906250,0.0,175430.843750,268782.625000,...,442658.781250,0.0,465458.937500,370437.156250,0.0,0.0,472186.937500,0.0,0.0,466427.312500
1,rabggta,20101.933594,62892.046875,0.000000,103993.031250,0.0,55333.929688,0.0,30097.763672,48279.539062,...,79100.062500,0.0,82092.179688,65288.472656,0.0,0.0,83334.328125,0.0,0.0,81504.468750
2,ywhae,6624.807129,14475.721680,0.000000,24332.548828,0.0,13422.669922,0.0,9587.588867,10549.949219,...,18221.716797,0.0,19292.927734,14790.755859,0.0,0.0,19355.056641,0.0,0.0,19177.373047
3,kat2b,4142.221680,8753.067383,0.000000,14703.916016,0.0,8161.906250,0.0,5902.970215,6346.573730,...,11016.152344,0.0,11690.555664,8958.273438,0.0,0.0,11691.131836,0.0,0.0,11626.661133
4,smad5,1322.737061,2288.863037,26.059177,3835.314453,0.0,2206.577148,0.0,1791.832397,1568.654663,...,2822.048828,0.0,3072.452148,2335.262207,0.0,0.0,3047.292969,0.0,0.0,3087.742920


In [22]:
header_of_MLP=['tag_'+str(i) for i in range(feature_set_MLP.shape[1]-1)]
features_MLP=np.array(feature_set_MLP[header_of_MLP])
gene_MLP=feature_set_MLP['gene_name']
print(features_MLP.shape)
print(len(gene_MLP))

(4655, 100)
4655


In [23]:
dictionary_MLP={}
u=0
for gn in gene_MLP:
    dictionary_MLP[gn]=features_MLP[u]
    u=u+1

In [24]:
header_of_LSTM=['tag_'+str(i) for i in range(feature_set_LSTM.shape[1]-1)]
features_LSTM=np.array(feature_set_LSTM[header_of_LSTM])
gene_LSTM=feature_set_LSTM['gene_name']
print(features_LSTM.shape)
print(len(gene_LSTM))

(4275, 100)
4275


In [26]:
dictionary_LSTM={}
u=f=0
for gn in gene_LSTM:
    if gn in dictionary_LSTM.keys():
#         print(gn)
        f=f+1
    dictionary_LSTM[gn]=features_LSTM[u]
    u=u+1
print(f)

494


In [27]:
print(len(dictionary_LSTM.keys()))

3781


In [28]:
fil=open('../../Multi-modality/Model/BCLL/uni model/MLP/label0.txt','r')
tmp=list()
for line in fil:
	tmp.append(int(line))

label_BCLL=np.array(tmp)
print(label_BCLL.shape)

(4655,)


In [33]:
class Sequenceloader(data.Dataset):
    def __init__(self,GN,Feat,label):
        self.gene_names=GN
        self.features_mlp=Feat
        self.label=label
        self.coincdgene_name=[]
        self.coincidfeature_MLP=[]
        self.coincidfeature_LSTM=[]
        self.coincidlabel=[]
        for i in range(len(self.gene_names)):
            u=self.gene_names[i]
            if u in dictionary_LSTM.keys():
                
                if np.array(self.label[i])==0:
                    ch=60
                elif np.array(self.label[i])==1:
                    ch=8
                else:
                    ch=1
                    
                for jj in range(ch):
                    self.coincdgene_name.append(u)
                    self.coincidfeature_MLP.append(self.features_mlp[i])
                    self.coincidfeature_LSTM.append(dictionary_LSTM[u])
                    self.coincidlabel.append(self.label[i])
        
        
    
    def __len__(self):
#         print(len(self.dataset))
        return len(self.coincdgene_name)       

    def __getitem__(self, index):
         return np.array(self.coincidfeature_MLP[index]),np.array(self.coincidfeature_LSTM[index]),np.array(self.coincidlabel[index])
#         print(self.dataset['gen_name'][index])          
#         try:
           
#         except :
            
total_set=Sequenceloader(gene_MLP,features_MLP,label_BCLL)  
a=b=c=0
for x,y,z in total_set:
    if(z==0):
        a=a+1
    elif z==1:
        b=b+1
    else:
        c=c+1
print(a,b,c)
    

3360 3504 3781


In [42]:
batch_size = 4
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(total_set)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(total_set, batch_size=batch_size, 
                                           sampler=train_sampler,drop_last=True)
validation_loader = torch.utils.data.DataLoader(total_set, batch_size=batch_size,
                                                sampler=valid_sampler,drop_last=True)


In [43]:
for a,b,c in train_loader:
    print(a.shape,b.shape,c.shape)

torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4,

torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4,

torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4,

torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4,

torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4,

torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4,

In [44]:
def test(test_loader,model):

    total_imgs=0;
    total_corrects=0
    u=0
    nb_classes=3
    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    for i1,i2,label in test_loader:
                

        output=model(i1.to(device).float(),i2.to(device).float())
        total_imgs=total_imgs+label.shape[0]
        z=torch.max(output,1)[1]==label.to(device)
        _, preds = torch.max(output, 1)
#         print(output.shape)
 
        num_corrects=torch.sum(z)
        total_corrects=total_corrects+num_corrects
        for t, p in zip(label.view(-1), preds.view(-1)):
            confusion_matrix[t.long(), p.long()] += 1


        u=u+1
    
    
    print(confusion_matrix)
    return(total_corrects,total_imgs)

In [45]:
device='cuda:7'
model_lstmXMLP=Encoder(2,100,4,300,300,True,4).to(device) #1)mlp 2lstm
uz=torch.rand(4, 100).to(device)
vz=torch.rand(4,100).to(device)
model_lstmXMLP(vz,uz).shape


torch.Size([4, 3])

In [46]:

#model_lstmXMLP.load_state_dict(torch.load(Path('1ANKIT_ILD COMBO_LSTMXMLP.pt')))
optim_params = [
    {'params': model_lstmXMLP.parameters(), 'lr': 0.0001}
]
optimizer = torch.optim.Adam(optim_params)
criterion = nn.CrossEntropyLoss()

In [47]:
epoch=10000
for i in range(epoch):
    torch.save(model_lstmXMLP.state_dict(), '1ANKIT_BCLL COMBO_LSTMXMLP.pt')
    print("Accuracy-"+str(test(validation_loader,model_lstmXMLP)))
    total_loss=0
    for inp1,inp2,lab in train_loader:

        

        output=model_lstmXMLP(inp1.to(device).float(),inp2.to(device).float())

        loss_batch=criterion(output,lab.to(device))/4
        optimizer.zero_grad()  
        loss_batch.backward()
        optimizer.step()
        total_loss+=loss_batch
       
    print(total_loss)
    
    
        
        

tensor([[ 55., 310., 326.],
        [173., 223., 296.],
        [570.,  86.,  89.]])
Accuracy-(tensor(367, device='cuda:7'), 2128)
tensor(195.8817, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[657.,  19.,  14.],
        [ 21., 627.,  44.],
        [ 15.,  44., 687.]])
Accuracy-(tensor(1971, device='cuda:7'), 2128)
tensor(145.8026, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[658.,  16.,  17.],
        [ 25., 608.,  58.],
        [ 14.,  45., 687.]])
Accuracy-(tensor(1953, device='cuda:7'), 2128)
tensor(134.6653, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[658.,  17.,  16.],
        [ 12., 636.,  43.],
        [ 23.,  46., 677.]])
Accuracy-(tensor(1971, device='cuda:7'), 2128)
tensor(131.6991, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[653.,  18.,  19.],
        [ 10., 642.,  40.],
        [ 10.,  43., 693.]])
Accuracy-(tensor(1988, device='cuda:7'), 2128)
tensor(126.8707, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[650.,  23.,  18.],
        [ 10., 646.,  3

tensor(90.7524, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[677.,   7.,   7.],
        [ 15., 640.,  36.],
        [  9.,  44., 693.]])
Accuracy-(tensor(2010, device='cuda:7'), 2128)
tensor(93.0257, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[674.,   4.,  13.],
        [ 16., 647.,  29.],
        [  9.,  37., 699.]])
Accuracy-(tensor(2020, device='cuda:7'), 2128)
tensor(90.8358, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[675.,   6.,  10.],
        [  9., 647.,  36.],
        [ 11.,  41., 693.]])
Accuracy-(tensor(2015, device='cuda:7'), 2128)
tensor(92.3647, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[671.,  12.,   8.],
        [  6., 649.,  37.],
        [ 16.,  41., 688.]])
Accuracy-(tensor(2008, device='cuda:7'), 2128)
tensor(88.3310, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[674.,   8.,   9.],
        [  9., 654.,  29.],
        [ 13.,  45., 687.]])
Accuracy-(tensor(2015, device='cuda:7'), 2128)
tensor(91.1174, device='cuda:7', grad_fn=<AddBackward0>

KeyboardInterrupt: 

In [ ]:
tensor([[675.,  11.,   5.],
        [  9., 649.,  33.],
        [ 12.,  37., 697.]])
Accuracy-(tensor(2021, device='cuda:7'), 2128)